In [ ]:
# Manipulação de dados
import duckdb
import pandas as pd

# Tratamento de dados
from scipy import stats
import numpy as np

# Visualização
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# Algoritmos
from xgboost import XGBRegressor, XGBClassifier

# Métricas de validação
from metrics import MAE,RMSE,R2_determinacao,PSC_A,PCC_A,PMC_A

# Salvar e importar modelos e métricas
import pickle
import json

# Configurações de bibliotecas
pd.set_option('display.max_columns',None)

# Importando o database de dados de modelagem
final_db = duckdb.connect(database='final_db')

# Defininindo os tipos de variáveis presentes no dataset
n_vizinhas_disponiveis = 10

list_key = ['id_estacao','dt_medicao']

list_features_geoespaciais = ['latitude','longitude','vl_declividade','vl_altitude','vl_distancia_oceano','vl_aspecto_relevo']

list_features_estacoes_temperatura = ['vl_temperatura_maxima','vl_temperatura_media','vl_temperatura_minima']
list_features_estacoes_umidade = ['vl_umidade_relativa_maxima','vl_umidade_relativa_media','vl_umidade_relativa_minima']
list_features_estacoes_vento = ['vl_velocidade_vento_2m_maxima','vl_velocidade_vento_2m_media','vl_velocidade_vento_10m_media']
list_features_estacoes = list_features_estacoes_temperatura + list_features_estacoes_umidade + list_features_estacoes_vento

list_features_chirps = ['vl_precipitacao_chirps']
list_features_cpc = ['vl_precipitacao_cpc','vl_temperatura_maxima_cpc','vl_temperatura_minima_cpc']
list_features_gpm_final_run = ['vl_precipitacao_gpm_final_run']
list_features_gpm_late_run = ['vl_precipitacao_gpm_late_run']
list_features_power = ['vl_precipitacao_power','vl_temperatura_maxima_2m_K_power','vl_temperatura_media_2m_K_power','vl_temperatura_minima_2m_K_power','vl_umidade_relativa_2m_power','vl_pressao_nivel_superficie_power','vl_irradiancia_allsky_power','vl_direcao_vento_10m_power','vl_direcao_vento_2m_power','vl_temperatura_orvalho_2m_K_power','vl_vento_10m_power','vl_vento_medio_2m_power','vl_vento_maximo_2m_power','vl_vento_maximo_10m_power']
list_features_produtos = list_features_chirps + list_features_cpc + list_features_gpm_final_run + list_features_gpm_late_run + list_features_power

list_vizinhas_aux = sum([[f'vl_correlacao_estacao_vizinha_{i}',f'pct_intersecao_precipitacao_vizinha_{i}',f'vl_distancia_km_vizinha_{i}',f'vl_prioridade_vizinha_{i}',f'vl_precipitacao_vizinha_{i}']for i in range(1, n_vizinhas_disponiveis+1)], [])
list_features_vizinhas = [f'vl_precipitacao_vizinha_{i}' for i in range(1,n_vizinhas_disponiveis+1)]

list_features_target = ['vl_precipitacao','vl_precipitacao_log']

list_split_column = ['percentil_temporal']

# MODELOS 1

In [10]:
print("""Os modelos 1 são os mais simples. Esses modelos são compostos por um único algoritmo de regressão, sempre usando XGBoost. A partição já é a padrão, de 70-30 por estação, e não há tentativas 
de modificação da target\n
> Sem pré modelo de classificação
> Sem modelo especializado
> Apenas algoritmo de XGBoost
> Partição padrão
> Sem modificações da target
> Testando diferentes combinações de variáveis explicativas
""")

Os modelos 1 são os mais simples. Esses modelos são compostos por um único algoritmo de regressão, sempre usando XGBoost. A partição já é a padrão, de 70-30 por estação, e não há tentativas 
de modificação da target

> Sem pré modelo de classificação
> Sem modelo especializado
> Apenas algoritmo de XGBoost
> Partição padrão
> Sem modificações da target
> Testando diferentes combinações de variáveis explicativas



In [75]:
def ImportBase_Modelo1(final_db=final_db,table_name='abt_base'):
    df = final_db.execute(f"""
    SELECT
        *
    FROM {table_name}
    """).fetch_df()

    return df

def SplitTreinoTeste_Modelo1(df_base,pct_split,coluna_percentil_temporal='percentil_temporal'):
  df_treino = df_base.loc[df_base[coluna_percentil_temporal]<=pct_split]
  df_teste = df_base.loc[df_base[coluna_percentil_temporal]>pct_split]
  return df_treino,df_teste

def PrepararBaseTreino(df_treino,list_features,target):
   df_X_treino = df_treino.loc[:,list_features]
   df_y_treino = df_treino.loc[:,[target]]
   return df_X_treino,df_y_treino

def TreinarAlgoritmo_Modelo1(df_X_treino,df_y_treino,Algoritmo):
   alg = Algoritmo()
   alg.fit(df_X_treino,df_y_treino)
   return alg

def RealizarPredicaoTeste(df_test,list_features,target,modelo,modelo_number='1_1'):
   df_X_test = df_test[list_features]
   df_y_pred = modelo.predict(df_X_test)
   df_validacao = df_test.copy()
   df_validacao[f'{target}_modelo_{modelo_number}'] = df_y_pred
   return df_validacao

def CalcularMetricasTeste(df_validacao,target,modelo_number,psc_a_max_chuva,pcc_a_erro,pmc_a_erro,pmc_a_min_chuva):
   metricas_modelo = {
      'MAE':MAE(df_validacao[target],df_validacao[f'{target}_modelo_{modelo_number}']),
      'RMSE':RMSE(df_validacao[target],df_validacao[f'{target}_modelo_{modelo_number}']),
      'R2':R2_determinacao(df_validacao[target],df_validacao[f'{target}_modelo_{modelo_number}']),
      'PSC_A':PSC_A(df_validacao[target],df_validacao[f'{target}_modelo_{modelo_number}'],psc_a_max_chuva),
      'PCC_A':PCC_A(df_validacao[target],df_validacao[f'{target}_modelo_{modelo_number}'],pcc_a_erro),
      'PMC_A':PMC_A(df_validacao[target],df_validacao[f'{target}_modelo_{modelo_number}'],pmc_a_erro,pmc_a_min_chuva)
   }

   return metricas_modelo

def SalvarValidacaoModeloMetricas_Modelo1(df_validacao,modelo_1_1,metricas_modelo_1_1,target,modelo_number,final_db=final_db):

    df_validacao_key_target_pred = df_validacao[list_key+[target,f'{target}_modelo_{modelo_number}']]

    final_db.execute(
    f"""
    CREATE OR REPLACE TABLE tb_validacao_modelo_{modelo_number} AS (
    SELECT * FROM df_validacao_key_target_pred)
    """)

    with open(f'modelos_finais/modelo_{modelo_number}.pkl','wb') as f:
        pickle.dump(modelo_1_1,f)

    with open(f'modelos_finais/metricas_{modelo_number}.json', 'w') as f:
        json.dump(metricas_modelo_1_1, f, indent=4)

    pass


## Modelo 1.1

In [ ]:
modelo_number = '1_1'

# Parâmetros de treinamento
pct_train_test_split = 0.7
target = 'vl_precipitacao'
list_features_modelo_1_1 = list_features_geoespaciais + list_features_estacoes
Algoritmo = XGBRegressor

# Parâmetros de métricas
psc_a_max_chuva = 1
pcc_a_erro = 1
pmc_a_erro = 5
pmc_a_min_chuva = 20

# Preparando base
df_base = ImportBase_Modelo1()
df_treino,df_teste = SplitTreinoTeste_Modelo1(df_base,pct_train_test_split)
df_X_treino,df_y_treino = PrepararBaseTreino(df_treino,list_features_modelo_1_1,target)

# Treinando o modelo
modelo_1_1 = TreinarAlgoritmo_Modelo1(df_X_treino,df_y_treino,Algoritmo)

# Validando
df_validacao = RealizarPredicaoTeste(df_teste,list_features_modelo_1_1,target,modelo_1_1,modelo_number)

# Calculando Métricas
metricas_modelo_1_1 = CalcularMetricasTeste(df_validacao,target,modelo_number,psc_a_max_chuva,pcc_a_erro,pmc_a_erro,pmc_a_min_chuva)

# Salvando modelo, base de validação e métricas
SalvarValidacaoModeloMetricas_Modelo1(df_validacao,modelo_1_1,metricas_modelo_1_1,target,modelo_number,final_db=final_db)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))